In [1]:
import pandas as pd 
from openai import OpenAI
import pandas as pd
import os
import json
from sklearn import model_selection
import sys
sys.path.insert(0,'/users/sgdbareh/volatile/ECHR_Importance')
from API_key import openai_key
from GPT_Experiments import Experiment_2, save_file, generate_example_candidates, create_examples
from optparse import OptionParser


In [5]:

JSON_SCHEMAS = [{"Case Importance":"string (select one of: key_case, 1, 2, 3)", "Reasoning":"string (give your reason for the importance)" }
                ]

PARAMETERS = {'schema':[JSON_SCHEMAS[0]],
              'zero_shot':[True,False],
              'text':[1,2,3],
              'reasoning':[False]
              }

#connect to api key
client = OpenAI(api_key=openai_key)

# Load the data
data = pd.read_pickle('/users/sgdbareh/volatile/ECHR_Importance/VectorDB/test.pkl')

train_data = pd.read_pickle('/users/sgdbareh/volatile/ECHR_Importance/VectorDB/train.pkl')

#set param grid search
grid = model_selection.ParameterGrid(PARAMETERS)

for params in grid:

    match params['text']:
        case 1:
            content = 'subject_matter'
        case 2:
            content = 'questions'
        case 3:
            content = 'both'

    exp_2 = Experiment_2(data,content=content,grand_chamber=False,reasoning=params['reasoning'])
    #set examples
    example_candidates = generate_example_candidates(train_data,keyword='importance',labels=[1,2,3,4])
    examples = create_examples(train_data,text=params['text'],importance=True)
    #create batch files for given prompt + parameters
    output = exp_2.run_async(schema=params['schema'],zero_shot=params['zero_shot'],text=params['text'],examples=examples,info=True,temperature=0,max_tokens=1500)
    #save batch file

    print(example_candidates)
    break
    #save_file(output,filepath='./Batches/experiment_2_CoT',batch_name='experiment_2_CoT',experiment = 2, zero_shot=params['zero_shot'],text=params['text'],schema=params['schema'],identifier=f'{params['reasoning']}_REASONING')


       Filename                                          Questions  \
40   001-110572  1. Was the investigation concerning the applic...   
470  001-146906  1. Has the application been introduced in comp...   
819  001-182690  1. Did the applicants exhaust the available do...   
320  001-127383  1. Did the application of paragraph 3(2)(b) of...   

                                        Subject Matter     appno  \
40   The applicant, Ms Y., is a Slovenian national ...  41107/10   
470  The applicant, Ms R.B., is a Hungarian nationa...  64602/12   
819  The application concerns the intended removal ...   5115/18   
320  The applicant, Mr Javad Nasseri, is an Afghan ...  24239/09   

                        source_file    doc_date importance  \
40         pruned_CHAMBER_meta.json  2015-05-28          1   
470        pruned_CHAMBER_meta.json  2016-04-12          2   
819        pruned_CHAMBER_meta.json  2020-02-20          3   
320  pruned_ADMISSIBILITY_meta.json  2015-10-13          4  

,Filename,Questions,Subject Matter,appno,source_file,doc_date,importance,keywords_art_3,keywords_art_3_text,Subj_Count,new_list_length,extracted_apps,new_list_length_2
40,001-110572,1. Was the investigation concerning the applic...,"The applicant, Ms Y., is a Slovenian national ...",41107/10,pruned_CHAMBER_meta.json,2015-05-28,1,"[350, 596]","(Art. 3) Prohibition of torture, (Art. 3) Effe...",3556,20,"[38281/08, 39272/98, 49669/07, 55523/00, 26828...",11
470,001-146906,1. Has the application been introduced in comp...,"The applicant, Ms R.B., is a Hungarian nationa...",64602/12,pruned_CHAMBER_meta.json,2016-04-12,2,"[350, 90, 596, 633]","(Art. 3) Prohibition of torture, (Art. 3) Degr...",679,32,"[43393/98, 47159/08, 2351/06, 13624/03, 26827/...",21
819,001-182690,1. Did the applicants exhaust the available do...,The application concerns the intended removal ...,5115/18,pruned_CHAMBER_meta.json,2020-02-20,3,"[350, 620]","(Art. 3) Prohibition of torture, (Art. 3) Expu...",274,13,"[40081/14, 59166/12, 46221/99, 43611/11, 27765...",8
320,001-127383,1. Did the application of paragraph 3(2)(b) of...,"The applicant, Mr Javad Nasseri, is an Afghan ...",24239/09,pruned_ADMISSIBILITY_meta.json,2015-10-13,4,[350],(Art. 3) Prohibition of torture,3077,8,"[71398/12, 28361/12, 21662/10, 32733/08, 30696...",8
